In [1]:
import pandas as pd
import numpy as np

In [2]:

data_dir = 'data/'

sample_dim = 100000

reviews_raw = pd.read_csv(data_dir + '/reviews.csv').head(sample_dim)
reviews_raw.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2737,24343517,2014-12-26,24333039,Raphaell,"This spot was in a great and nice area,walking..."
1,2737,24966779,2015-01-04,10493776,Ia,"Elif was really very helpful, while arranging ..."
2,2737,29770086,2015-04-12,29630601,Julia,"Elif's flat is spacious, clean, and had everyt..."
3,2737,30477570,2015-04-22,31215535,Sevil,I had a wonderful time staying with Elif and h...
4,2737,31659336,2015-05-08,1959813,Suzy,"Elif was a brilliant host, with an exceptional..."


In [3]:
reviews_raw.columns.to_list()

['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments']

### Date in formato datetime

In [4]:
reviews = reviews_raw.assign(date = pd.to_datetime(reviews_raw['date']))
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2737,24343517,2014-12-26,24333039,Raphaell,"This spot was in a great and nice area,walking..."
1,2737,24966779,2015-01-04,10493776,Ia,"Elif was really very helpful, while arranging ..."
2,2737,29770086,2015-04-12,29630601,Julia,"Elif's flat is spacious, clean, and had everyt..."
3,2737,30477570,2015-04-22,31215535,Sevil,I had a wonderful time staying with Elif and h...
4,2737,31659336,2015-05-08,1959813,Suzy,"Elif was a brilliant host, with an exceptional..."


In [5]:
data_dir = 'data/'

calendar_raw = pd.read_csv(data_dir + '/calendar.csv').head(sample_dim)
calendar = calendar_raw.assign(date = pd.to_datetime(calendar_raw['date']))
calendar.head()


,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,158668,2021-02-13,t,$40.00,$40.00,2.0,365.0
1,78314,2021-02-13,f,$69.00,$69.00,1.0,99.0
2,78314,2021-02-14,f,$69.00,$69.00,1.0,99.0
3,78314,2021-02-15,f,$69.00,$69.00,1.0,99.0
4,78314,2021-02-16,f,$69.00,$69.00,1.0,99.0


# Data prepation

## Creare le variabli di aggregazione mese e anno

In [13]:
%timeit
reviews['year'] = reviews['date'].dt.year
reviews['month'] = reviews['date'].dt.month
reviews = reviews.sort_values(['year', 'month'], ascending=False)
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,year,month
4310,50471,729150541,2021-02-06,8582830,Anders,"Lovely place, super quiet and great atmosphere :)",2021,2
38546,335009,728807735,2021-02-04,95567665,Soraja,It was ideal. The Host is very responsive and ...,2021,2
66801,639627,728358403,2021-02-01,64638603,Nathalie,Massimo is kind and prompt in responding. The ...,2021,2
73665,721874,728694435,2021-02-03,210113502,Lubna,I liked my stay in Sabrina’s place. It was a w...,2021,2
88513,831199,728361293,2021-02-01,358777714,Carla,Che dire il sig. Pietro e la moglie persone mo...,2021,2


In [8]:
%timeit
calendar['year'] = pd.DatetimeIndex(calendar['date']).year
calendar['month'] = pd.DatetimeIndex(calendar['date']).month
calendar = calendar.sort_values(['year', 'month'], ascending=False)
calendar.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,year,month
398,2737,2022-02-01,t,$18.00,$18.00,7.0,1125.0,2022,2
399,2737,2022-02-02,t,$18.00,$18.00,7.0,1125.0,2022,2
400,2737,2022-02-03,t,$18.00,$18.00,7.0,1125.0,2022,2
401,2737,2022-02-04,t,$30.00,$30.00,7.0,1125.0,2022,2
402,2737,2022-02-05,t,$30.00,$30.00,7.0,1125.0,2022,2


## Preparare il prezzo come numero

In [14]:
# PREVIOUS WAY
# @np.vectorize
# def remove_dollar(label: str):
#   return float(label.replace('$','').replace(',',''))

calendar_2 = calendar.assign(
        price          = pd.to_numeric(calendar.price.str.replace('$','').str.replace(',','')),
        adjusted_price = pd.to_numeric(calendar.adjusted_price.str.replace('$','').str.replace(',','')),
    )
calendar_2['price'].head()

398    18.0
399    18.0
400    18.0
401    30.0
402    30.0
Name: price, dtype: float64

In [15]:
calendar_2.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,year,month
398,2737,2022-02-01,t,18.0,18.0,7.0,1125.0,2022,2
399,2737,2022-02-02,t,18.0,18.0,7.0,1125.0,2022,2
400,2737,2022-02-03,t,18.0,18.0,7.0,1125.0,2022,2
401,2737,2022-02-04,t,30.0,30.0,7.0,1125.0,2022,2
402,2737,2022-02-05,t,30.0,30.0,7.0,1125.0,2022,2


## Valori logici


In [16]:
calendar_2['available'] = calendar_2.available.map({
    't': True,
    'f': False
})

## 

# Analisi delle Attività

Gli ospiti che usano Airbnb possono lasciare una recensione dopo il loro pernottamento. Le recensioni possono essere usate come indicatori dell'attività e del tasso di attività.

La durata minima di pernottamento, il prezzo e il numero di recensioni sono stati usati come indicatori per stimare il tasso di occupazione, il numero di notti per anno e il rendita mensile di ogni annuncio.

Qual è il rapporto tra la rendita realizzata attraverso Airbnb e i prezzi degli affitti residenziali a lungo termine?

Il numero delle notti prenotate impedisce l'uso residenziale degli alloggi messi in affitto?

Qual è il rapporto tra la rendita realizzata attraverso AIRBNB e i prezzi degli affitti residenziali a lungo termine?




## Tasso di occupazione

### Creo le variabili che mi servono


In [17]:
# Transformations
calendar_2['available_prob'] = calendar_2.available.map({
    True: 1,
    False: 0
})

calendar_2['engaged_prob'] = 1 - calendar_2['available_prob']
calendar_2


,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,year,month,available_prob,engaged_prob
398,2737,2022-02-01,True,18.0,18.0,7.0,1125.0,2022,2,1,0
399,2737,2022-02-02,True,18.0,18.0,7.0,1125.0,2022,2,1,0
400,2737,2022-02-03,True,18.0,18.0,7.0,1125.0,2022,2,1,0
401,2737,2022-02-04,True,30.0,30.0,7.0,1125.0,2022,2,1,0
402,2737,2022-02-05,True,30.0,30.0,7.0,1125.0,2022,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...
99868,263210,2021-02-24,False,80.0,80.0,3.0,1125.0,2021,2,0,1
99869,263210,2021-02-25,False,80.0,80.0,3.0,1125.0,2021,2,0,1
99870,263210,2021-02-26,False,80.0,80.0,3.0,1125.0,2021,2,0,1
99871,263210,2021-02-27,False,80.0,80.0,3.0,1125.0,2021,2,0,1


In [108]:
# Transformations
# calendar_2\
#     .assing(
#         available_prob = calendar_2.available.map({
#             True: 1,
#             False: 0
#         }),
#         engaged_prob = 1 - calendar_2['available_prob']
#     )

AttributeError: 'DataFrame' object has no attribute 'assing'

### tasso medio di occupazione mensile e per alloggio:

In [18]:
engagement_tbl = calendar_2\
    .groupby(['year', 'month', 'listing_id'])\
    .engaged_prob\
    .mean()\
    .reset_index()\
    .sort_values(['engaged_prob', 'listing_id', 'year', 'month'], ascending=False)
engagement_tbl

,year,month,listing_id,engaged_prob
2197,2021,9,312492,1.0
1923,2021,8,312492,1.0
1649,2021,7,312492,1.0
546,2021,3,263210,1.0
270,2021,2,263210,1.0
...,...,...,...,...
1102,2021,6,2737,0.0
827,2021,5,2737,0.0
552,2021,4,2737,0.0
276,2021,3,2737,0.0


In [21]:
engagement_tbl = calendar_2\
    .groupby(['listing_id'])\
    .engaged_prob\
    .mean()\
    .reset_index()\
    .sort_values(['engaged_prob', 'listing_id'], ascending=False)
engagement_tbl

,listing_id,engaged_prob
270,263210,1.0
239,230555,1.0
219,210511,1.0
204,192711,1.0
195,181189,1.0
...,...,...
12,20699,0.0
6,15409,0.0
2,3079,0.0
1,2903,0.0


In [27]:
### tasso medio di occupazione 1

In [23]:
engagement_tbl = calendar_2\
    .groupby(['year', 'month'])\
    .engaged_prob\
    .mean()\
    .reset_index()\
    .sort_values(['engaged_prob','year', 'month'], ascending=False)\
    .engaged_prob\
    .mean()
engagement_tbl

0.26314159194727194

### tasso medio di occupazione 2

In [24]:
calendar_2.engaged_prob.mean()

0.25863

### Notti per anno ???

In [31]:
calendar_2

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,year,month,available_prob,engaged_prob
398,2737,2022-02-01,True,18.0,18.0,7.0,1125.0,2022,2,1,0
399,2737,2022-02-02,True,18.0,18.0,7.0,1125.0,2022,2,1,0
400,2737,2022-02-03,True,18.0,18.0,7.0,1125.0,2022,2,1,0
401,2737,2022-02-04,True,30.0,30.0,7.0,1125.0,2022,2,1,0
402,2737,2022-02-05,True,30.0,30.0,7.0,1125.0,2022,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...
99868,263210,2021-02-24,False,80.0,80.0,3.0,1125.0,2021,2,0,1
99869,263210,2021-02-25,False,80.0,80.0,3.0,1125.0,2021,2,0,1
99870,263210,2021-02-26,False,80.0,80.0,3.0,1125.0,2021,2,0,1
99871,263210,2021-02-27,False,80.0,80.0,3.0,1125.0,2021,2,0,1


In [37]:
# tutto sbagliato
calendar_2[calendar_2.available]\
  .groupby(['year'], dropna=False)\
  .listing_id.count()\
  .sort_index(ascending=False)\
  .reset_index()\
  .rename(columns={'id': 'count'})

,year,listing_id
0,2022,8320
1,2021,65817


neanche questo ha senso

In [93]:
# ????
review_count = reviews\
  .groupby(['year'], dropna=False)\
  .id.count()\
  .sort_index(ascending=False)\
  .reset_index()\
  .rename(columns={'id': 'count'})

# review_count['count_perc'] = (review_count['count'] / review_count['count'].sum() * 100).round(2)
review_count['nights_per_year'] = (review_count['count'] / 365).round(2)

review_count['nights_per_year'].mean()

19.57

### Prezzo a notte medio (nemmeno questo ha senso)

In [94]:
calendar_2.price.mean()

100.03733

In [95]:
calendar_2.loc[calendar_2.engaged_prob == 1].price.mean()

102.4371882612226

### Conteggio Recensioni

In [39]:
# Total reviews:
reviews.id.count()

100000

### Recensioni / annunci / mese

In [46]:

review_count = reviews\
  .groupby(['year', 'month', 'listing_id'], dropna=False)\
  .id.count()\
  .sort_index(ascending=False)\
  .reset_index()\
  .rename(columns={'id': 'count'})

review_count.head()


,year,month,listing_id,count
0,2021,2,865289,1
1,2021,2,831199,1
2,2021,2,721874,1
3,2021,2,639627,1
4,2021,2,335009,1


In [44]:
# review_count['count_perc'] = (review_count['count'] / review_count['count'].sum() * 100).round(2)
# review_count['nights_per_year'] = (review_count['count'] / 365).round(2)

review_count['count'].mean()

2.7159152634437804

### Fatturato al mese stimato (probabilmente sbagliato)

In [47]:
mean_day_price_engaged = calendar_2\
    .assign(day_income = calendar_2.price * calendar_2.engaged_prob )\
    .day_income\
    .mean()
mean_day_price_engaged * 30
    


794.7999

# GC


In [24]:
import gc

calendar_raw = None
reviews_raw = None
review_count = None
calendar = None

gc.collect()

40